# Setup

In [96]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai  # type: ignore
from google.cloud import documentai_v1beta3 as documentai_beta
from typing import Iterator, Optional, Sequence, Tuple
import os
from google.protobuf import timestamp_pb2
from datetime import datetime
import pandas as pd

In [97]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'docai-demo-387616'

In [108]:
#Setup
LOCATION = 'us' # Format is 'us' or 'eu'
PROCESSOR_ID = 'f75fbb5887c807f4'
#PROCESSOR_VERSION_ID = 'pretrained-invoice-v1.3-2022-07-15'
API_LOCATION = "us"  # Choose "us" or "eu"

DATASET = 'invoice_demo'
TABLE_1 = 'invoice_parser_eval_stats'
TABLE_2 = 'invoice_parser_eval_stats_per_doc'

# SDK version to pull evals and load to BQ

In [110]:
def get_client() -> documentai.DocumentProcessorServiceClient:
    client_options = {"api_endpoint": f"{API_LOCATION}-documentai.googleapis.com"}
    return documentai.DocumentProcessorServiceClient(client_options=client_options)


def get_parent(client: documentai.DocumentProcessorServiceClient) -> str:
    return client.common_location_path(PROJECT_ID, API_LOCATION)


def get_client_and_parent() -> Tuple[documentai.DocumentProcessorServiceClient, str]:
    client = get_client()
    parent = get_parent(client)
    return client, parent

In [111]:
def list_processors() -> Sequence[documentai.Processor]:
    client, parent = get_client_and_parent()
    response = client.list_processors(parent=parent)

    return response.processors

In [169]:
#list_processors()

In [113]:
def list_processor_versions(project_id: str, location: str, processor_id: str) -> None:
    # You must set the api_endpoint if you use a location other than 'us'.
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor
    # e.g.: projects/project_id/locations/location/processors/processor_id
    parent = client.processor_path(project_id, location, processor_id)

    # Make ListProcessorVersions request
    processor_versions = client.list_processor_versions(parent=parent)
    
    version_list = []

    # Print the processor version information
    for processor_version in processor_versions:
        version_list.append(client.parse_processor_version_path(
            processor_version.name
        )["processor_version"])
    return version_list
        

In [135]:
processor_versions = list_processor_versions(PROJECT_ID, LOCATION, PROCESSOR_ID)
processor_versions

['pretrained-invoice-v1.4-2022-10-21',
 'pretrained-invoice-v1.3-2022-07-15',
 'pretrained-invoice-v1.2-2022-02-18',
 'pretrained-invoice-v1.1-2021-04-09']

In [137]:
def list_evaluations(
    project_id: str, location: str, processor_id: str, processor_version_id: str
) -> None:
    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the processor version
    # e.g. `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    parent = client.processor_version_path(
        project_id, location, processor_id, processor_version_id
    )

    evaluations = client.list_evaluations(parent=parent)

    return evaluations



In [143]:
eval_list = []
for i in processor_versions:
    temp = [i, list_evaluations(PROJECT_ID, LOCATION, PROCESSOR_ID, i).evaluations[0].name[-16:]]
    eval_list.append(temp)
eval_list

[['pretrained-invoice-v1.4-2022-10-21', '4b836dc63859ebcd'],
 ['pretrained-invoice-v1.3-2022-07-15', 'e4abc501d57359e0'],
 ['pretrained-invoice-v1.2-2022-02-18', 'e0a21b6f6c7e4999'],
 ['pretrained-invoice-v1.1-2021-04-09', 'eb0747b09865f971']]

## Get metrics from evaluation run

In [117]:
def get_evaluation(
    project_id: str,
    location: str,
    processor_id: str,
    processor_version_id: str,
    evaluation_id: str,
) -> None:
    # You must set the api_endpoint if you use a location other than 'us', e.g.:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the evaluation
    # e.g. `projects/{project_id}/locations/{location}/processors/{processor_id}/processorVersions/{processor_version_id}`
    evaluation_name = client.evaluation_path(
        project_id, location, processor_id, processor_version_id, evaluation_id
    )
    # Make GetEvaluation request
    evaluation = client.get_evaluation(name=evaluation_name)

    name = evaluation.name
    create_time = evaluation.create_time
    document_counters = evaluation.document_counters
    all_metrics = evaluation.all_entities_metrics
    metrics = evaluation.entity_metrics

    return evaluation

In [173]:
eval_data = get_evaluation(PROJECT_ID, LOCATION, PROCESSOR_ID, eval_list[0][0], eval_list[0][1])


In [176]:
eval_data

name: "projects/716637756816/locations/us/processors/f75fbb5887c807f4/processorVersions/pretrained-invoice-v1.4-2022-10-21/evaluations/4b836dc63859ebcd"
create_time {
  seconds: 1685037145
  nanos: 65317000
}
entity_metrics {
  key: "amount_due"
  value {
  }
}
entity_metrics {
  key: "amount_paid_since_last_invoice"
  value {
  }
}
entity_metrics {
  key: "carrier"
  value {
  }
}
entity_metrics {
  key: "currency"
  value {
    confidence_level_metrics {
      metrics {
        precision: 1.0
        recall: 1.0
        f1_score: 1.0
        predicted_occurrences_count: 1
        ground_truth_occurrences_count: 1
        true_positives_count: 1
        total_documents_count: 1
        predicted_document_count: 1
        ground_truth_document_count: 1
      }
    }
    confidence_level_metrics {
      confidence_level: 0.009999999776482582
      metrics {
        precision: 1.0
        recall: 1.0
        f1_score: 1.0
        predicted_occurrences_count: 1
        ground_truth_occurr

In [191]:
tp = 0
fp = 0
fn = 0

for key in eval_data.entity_metrics.keys():
    for j in eval_data.entity_metrics[key].confidence_level_metrics:
        if j.metrics.true_positives_count:
            tp += j.metrics.true_positives_count
        if j.metrics.false_negatives_count:
            fn += j.metrics.false_negatives_count
        if j.metrics.false_positives_count:
            fp += j.metrics.false_positives_count
    for k in eval_data.entity_metrics[key].confidence_level_metrics_exact:
        if k.metrics.true_positives_count:
            tp += k.metrics.true_positives_count
        if k.metrics.false_negatives_count:
            fn += k.metrics.false_negatives_count
        if k.metrics.false_positives_count:
            fp += k.metrics.false_positives_count

precision = tp/(tp + fp)
print('precision: ' + str(precision))
recall = tp/(tp + fn)
print('recall: ' + str(recall))
f1 = (2*tp)/(2*tp + fp + fn)
print('f1: ' + str(f1))


precision: 0.65263644773358
recall: 0.41089108910891087


In [190]:
tp = 0
fp = 0
fn = 0

for key in eval_data.entity_metrics.keys():
    for j in eval_data.entity_metrics[key].confidence_level_metrics:
        if j.metrics.true_positives_count:
            tp += j.metrics.true_positives_count
        if j.metrics.false_negatives_count:
            fn += j.metrics.false_negatives_count
        if j.metrics.false_positives_count:
            fp += j.metrics.false_positives_count
    for k in eval_data.entity_metrics[key].confidence_level_metrics_exact:
        if k.metrics.true_positives_count:
            tp += k.metrics.true_positives_count
        if k.metrics.false_negatives_count:
            fn += k.metrics.false_negatives_count
        if k.metrics.false_positives_count:
            fp += k.metrics.false_positives_count

precision = tp/(tp + fp)
print('precision: ' + str(precision))
recall = tp/(tp + fn)
print('recall: ' + str(recall))

precision: 0.6974552309142319
recall: 0.38481539261570463


In [157]:
eval_df = pd.DataFrame(columns=['name', 'create_time', 'label', 'confidence_level', 'precision', 'recall', 'f1_score'])

for i in eval_list: 
    eval_data = get_evaluation(PROJECT_ID, LOCATION, PROCESSOR_ID, i[0], i[1])
    eval_name = eval_data.name
    eval_time = datetime.fromtimestamp(eval_data.create_time.timestamp())
    eval_time_formatted = eval_time.strftime('%Y-%m-%d %H:%M:%S')
    for key in eval_data.entity_metrics.keys():
        for j in eval_data.entity_metrics[key].confidence_level_metrics:
            row = [eval_name, eval_time_formatted, key, j.confidence_level, j.metrics.precision, j.metrics.recall, j.metrics.f1_score]
            eval_df = eval_df.append(pd.DataFrame([row], columns=['name', 'create_time', 'label', 'confidence_level', 'precision', 'recall', 'f1_score']), ignore_index=True)
        

In [158]:
#eval_df.info()

In [161]:
#eval_df.describe()
eval_df = eval_df[eval_df['f1_score'] >= 0]

In [163]:
eval_df[eval_df['name'] == 'projects/716637756816/locations/us/processors/f75fbb5887c807f4/processorVersions/pretrained-invoice-v1.1-2021-04-09/evaluations/eb0747b09865f971'].describe()

,confidence_level,precision,recall,f1_score
count,2525.000000,1003.000000,1717.000000,2525.000000
mean,0.500000,0.570088,0.356339,0.226278
std,0.291605,0.427350,0.438749,0.379035
min,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.000000,0.000000,0.000000
50%,0.500000,0.500000,0.000000,0.000000
75%,0.750000,1.000000,1.000000,0.428571
max,1.000000,1.000000,1.000000,1.000000


## Migrate DF to BQ

In [160]:
from google.cloud import bigquery

client = bigquery.Client()

table_id = f'{PROJECT_ID}.{DATASET}.{TABLE_2}'

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    eval_df, table_id, job_config=job_config
)
job.result()  # Wait for the job to complete.
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 8484 rows and 7 columns to docai-demo-387616.invoice_demo.invoice_parser_eval_stats_per_doc


# Using jsons downloaded from UI

In [194]:
eval_summ_df = pd.DataFrame()

for file in os.listdir("data_eval/"):
    if file.endswith(".json"):
        df = pd.read_json(str('data_eval/' + file))
        df = df.reset_index()
        df = df.rename(columns={'index': 'label'})
        df = df.drop('isFuzzyMatch', axis=1)
        df = pd.concat([df.drop(['metrics'], axis=1), df['metrics'].apply(pd.Series)], axis=1)
        df['id'] = df['processorName'].astype(str) + df['versionName'].astype(str) + df['createTime'].astype(str)

        eval_summ_df = pd.concat([eval_summ_df, df])
eval_summ_df = eval_summ_df.reset_index()

In [195]:
#eval_summ_df.head()

In [196]:
from google.cloud import bigquery

client = bigquery.Client()

table_id = f'{PROJECT_ID}.{DATASET}.{TABLE_1}'

job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.job.WriteDisposition.WRITE_TRUNCATE
)

job = client.load_table_from_dataframe(
    eval_summ_df, table_id, job_config=job_config
)
job.result()  # Wait for the job to complete.
table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 208 rows and 19 columns to docai-demo-387616.invoice_demo.invoice_parser_eval_stats
